# Regrouper les valeurs produites par Analog en a) lemmes, b) pos et faire un nouveau CSV

_Attention, il faut avoir réencodé le CSV produit par Analog en UTF-8 préalablement._

In [1]:
def compil_analog(chemin_entree, chemin_sortie):
    
    """
    Fonction récupérant un fichier CSV produit par Analog
    pour produire un nouveau CSV donnant aux informations
    la forme dans laquelle elles doivent être transcrites
    en XML-TEI.
    
    Attention : 
    
        - Analog produit un fichier en UTF-16, il faut donc
    le convertir en UTF-8 avant de pouvoir l'utiliser ici.
        - L'ordre et le nombre de colonnes est conditionné par
    les réglages faits sur Analog avant l'analyse des tokens.
    Les variables de la fonction sont donc conçues pour
    fonctionner avec un réglage particulier choisi pour
    le corpus du projet CONDÉ par le Dr Mathieu Goux.
        - À cause de problèmes avec les virgules, nous avions
    changé les séparateurs de colonne en points-virgules. Le
    CSV produit par ce script rétablit la virgule comme
    séparateur.
    
    :param chemin_entree: Le chemin interne du fichier CSV
    contenant l'analyse d'Analog à convertir.
    
    :param chemin_sortie: Le chemin interne du fichier CSV
    à créer, dans lequel seront stockées les informations
    une fois converties.
    
    """
    
    import csv
    import re
    
    # Le séparateur choisi pour séparer deux analyses concurrentes pour un même token.
    separateur = "|"
    
    # Le réglage choisi pour Analog ne produisant qu'un niveau de lemme, les deux formes
    # données sont identiques : on ne garde donc que la première. Cette RegEx permet de
    # sélectionner un couple de parenthèses et leur contenu.
    parentheses = re.compile(r"\([^\)]+\)")
    
    # Les colonnes du futur fichier CSV de sortie.
    colonnes = ["Numero de token", "Mot forme", "Validation automatique", "Lemme(s)", "POS"]
    
    # De quoi compter les tokens validés automatiquement par Analog :
    compteur_total = 0
    compteur_valides = 0
    
    
    # Lecture du fichier CSV d'Analog.
    with open(chemin_entree) as csv_a_lire:
        csv_lu = csv.DictReader(csv_a_lire, delimiter=";")
        
        # Ouverture du fichier CSV de sortie.
        with open(chemin_sortie, 'w') as csv_a_ecrire:
            a_ecrire = csv.DictWriter(csv_a_ecrire, fieldnames=colonnes)
            
            # Écriture des noms des colonnes en première ligne.
            a_ecrire.writeheader()
        
            # On boucle sur les lignes du CSV d'Analog.
            for mot in csv_lu:
                compteur_total += 1
                
                # On stocke l'identifiant du mot dans "num"
                # et on ajoute 1 car Analog indexe à partir de 0.
                num = int(mot["Mot n°"]) + 1
                
                # On stocke le mot-forme dans "forme".
                forme = mot["Forme rencontrée"]
                
                # Si Analog n'a pas vu d'ambiguïté, on renseigne
                # automatiquement les valeurs restantes.
                if mot["Mode Validation"] == "VA/DS":
                    compteur_valides += 1
                    validation = "yes"
                    lemma = mot["sous-type 1.1"]
                    pos = mot["Niveau 1"]
                    
                # Si Analog n'était pas sûr, on compile les différentes
                # valeurs possibles avant de les renseigner.
                else:
                    validation = "no"
                    lemma_list = []
                    pos_list = []
                    
                    # On boucle sur les noms de colonnes.
                    for colonne in mot.keys():
                        
                        # En excluant les deux ne correspondant pas à
                        # l'analyse d'Analog.
                        if colonne != "Mot n°" and colonne != "Forme rencontrée":
                            
                            # Si la colonne n'est pas vide dans la ligne courante.
                            if mot[colonne]:
                                
                                # On ajoute son contenu, avant les parenthèses,
                                # à la liste de lemmes possibles.
                                lemma_list.append(parentheses.sub("",mot[colonne]))
                                
                                # On ajoute son nom à la liste de POS possibles.
                                pos_list.append(colonne)
                    
                    # On joint les deux listes en séparant les possibilités
                    # par le pipe, défini en début de fonction.
                    if len(set(lemma_list)) == 1:
                        lemma = lemma_list[0]
                    else:
                        lemma = separateur.join(lemma_list)
                    """if re.fullmatch(lemmes_identiques, lemma):
                        lemma = lemma_list[0]"""
                        
                    pos = separateur.join(pos_list)
                
                # On crée une nouvelle ligne dans le fichier de sortie et
                # on y renseigne les informations compilées pour le mot courant.
                
                a_ecrire.writerow({
                    'Numero de token': num,
                    'Mot forme': forme,
                    'Validation automatique': validation,
                    'Lemme(s)': lemma,
                    'POS': pos
                })
    print(compteur_valides, "validés : ", round(compteur_valides * 100 / compteur_total, 1), "%")

In [2]:
# Utilisation de la fonction. Changer les chemins quand nécessaire.

compil_analog('/home/erminea/Documents/CONDE/Morisse-TS-XML/Morisse_Analog.csv',
             '/home/erminea/Documents/CONDE/Morisse-TS-XML/morisse_analog_compile.csv')

1370 validés :  27.2 %
